In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks/3-2/"

In [ ]:
#바이오 companyId와 마켓 데이터 불러오기
import pandas as pd
df = pd.read_parquet('./inner_bio_companyId.parquet')
indexes = list(df['companyId'])

import pyarrow.parquet as pq
import pandas as pd
filters = [('companyId', 'in', indexes)]
df_market = pd.read_parquet('./market', filters=filters, columns=['companyId', 'companyName', 'pricingDate', 'priceClose', 'adjustmentFactor'])
df_market['companyId'] = list(map(int, df_market['companyId']))

In [ ]:
df_market[df_market['companyId'] == 39414143]

In [ ]:
#바이오 예상치 데이터 불러오기
import pyarrow.parquet as pq
import pandas as pd
filters = [('companyId', 'in', indexes)]
df_estimates = pd.read_parquet('./estimates', filters=filters, columns=['companyId', 'fiscalYear', 'dataItemName', 'dataItemValue', 'periodTypeName', 'ISOCode', 'estimateScaleName'])
df_estimates

In [ ]:
df_estimates['companyId'] = list(map(int, df_estimates['companyId']))
df_estimates['dataItemValue'] = list(map(float, df_estimates['dataItemValue']))
df_estimates['fiscalYear'] = list(map(int, df_estimates['fiscalYear']))

pd.options.mode.chained_assignment = None
df_est = df_estimates.copy()

for i in range(len(df_est)):
  if df_est['estimateScaleName'][i] == 'Millions':
    df_est['dataItemValue'][i] = df_est['dataItemValue'][i] * 1_000_000
  elif df_est['estimateScaleName'][i] == 'Billions':
    df_est['dataItemValue'][i] = df_est['dataItemValue'][i] * 1_000_000_000
df_est.drop(['estimateScaleName'], axis=1, inplace=True)

#39414143 예상치 데이터가 -e^11이 나옴에 Outlier라고 판단하여 제거
df_est = df_est[df_est['companyId'] != 39414143]
df_est

#Estimates 데이터 항목 리스트
estimates_unique = df_est.groupby(['dataItemName', 'periodTypeName']).size().reset_index(name='freq')
estimates_unique.drop('freq', axis=1, inplace=True)

#얘네만 다룰거임
annual = estimates_unique[estimates_unique['periodTypeName'] == 'Annual']
annual_item = list(annual['dataItemName'])
annual_item

In [ ]:
#환율 데이터 불러오기 + 22년 ~ 23년까지 Currency별로 데이터프레임 정리
import pyarrow.parquet as pq
import pandas as pd
df_exchange = pd.read_parquet('./exchange')

df_ex = df_exchange.copy()
df_ex.sort_values('priceDate', ascending=True, inplace=True)
df_ex.set_index('priceDate', drop=True, inplace=True)

code = list(df_ex['ISOCode'].unique())
fx = pd.DataFrame()

for c in code:
  ex = df_ex[(df_ex['ISOCode'] == c)].copy()
  ex.rename(columns={'priceClose' : c}, inplace=True)
  ex.drop(['ISOCode'], axis=1, inplace=True)
  d = ex.loc['2022-09-22':'2023-09-22'].copy()
  d = d[~d.index.duplicated(keep='first')]
  d = d.apply(pd.to_numeric, errors='coerce')
  fx = pd.concat([fx, d], axis=1)

In [ ]:
#바이오 재무 데이터 불러오기

import pyarrow.parquet as pq
import pandas as pd
filters = [('companyId', 'in', indexes)]
df_finance = pd.read_parquet('./financials', filters=filters, columns=['companyId', 'periodTypeName', 'fiscalQuarter', 'fiscalYear', 'dataItemName', 'dataItemValue', 'ISOCode', 'unitTypeValue'])

#데이터 처리를 위해 필요한 칼럼들을 Int 형태로 변환
df_finance['companyId'] = list(map(int, df_finance['companyId']))
df_finance['dataItemValue'] = list(map(float, df_finance['dataItemValue']))
df_finance['fiscalYear'] = list(map(int, df_finance['fiscalYear']))
df_finance['unitTypeValue'] = list(map(int, df_finance['unitTypeValue']))

#주어진 UnitTypeValue를 Value에 곱하여 단위를 맞춰줌

df_finance['dataItemValue'] = df_finance['dataItemValue'] * df_finance['unitTypeValue']
df_finance.drop(['unitTypeValue'], axis=1, inplace=True)

##39414143 기업의 데이터가 Outlier라고 판단하여 제거
df_finance = df_finance[df_finance['companyId'] != 39414143]

#재무 데이터 중 고유 아이템을 그룹으로 묶어서 분류
financials_unique = df_finance.groupby(['dataItemName', 'periodTypeName']).size().reset_index(name='freq')
financials_unique.drop('freq', axis=1, inplace=True)
financials_unique

In [ ]:
#한 고유 데이터가 분기별, 연간별로 두 개로 나누어져서 존재함
#그 중에 분기별 데이터만 사용함
quarter = financials_unique[financials_unique['periodTypeName'] == 'Quarterly']
quarter_item = list(quarter['dataItemName'])
quarter_item

In [ ]:
#재무 데이터가 존재하는 기업이 많이 없는 경우가 있어 데이터 당 기업이 500개 이상인 기업에 대해서만 분석을 진행
financial_item = []
for item in quarter_item:
  example_feature = df_finance[(df_finance['dataItemName'] == item) & (df_finance['periodTypeName'] == 'Quarterly')]
  id = len(example_feature['companyId'].unique())
  if id > 500:
    financial_item.append(item)

print(len(financial_item))

In [ ]:
from tqdm import tqdm

In [ ]:
#마켓 데이터로부터 기업의 주가를 시계열로 데이터로 정리
df_price = df_market.copy()
df_price['price'] = df_price['priceClose'] * df_price['adjustmentFactor']
df_price['price'] = df_price['price'].apply(lambda x: round(x, 4))
df_price.drop(['priceClose', 'adjustmentFactor'], axis=1, inplace=True)

df_price['companyId'] = list(map(int, df_price['companyId']))
#unique_id = list(df_price['companyId'].unique())
unique_id = list(df_price['companyName'].unique())
market = pd.DataFrame()

#팩터 모델링을 위한 Training의 기간은 21년 9월 22일부터 22년 9월 22일까지
for id in tqdm(unique_id):
  ex = df_price[(df_price['companyName'] == id)].copy()
  ex.rename(columns={'price' : id}, inplace=True)
  ex.drop(['companyName'], axis=1, inplace=True)
  ex.sort_values(by='pricingDate', inplace=True)
  ex.set_index('pricingDate', inplace=True, drop=True)
  d = ex.loc['2022-09-22':'2023-09-22'].copy()
  d = d[~d.index.duplicated(keep='first')]
  d = d.apply(pd.to_numeric, errors='coerce')
  market = pd.concat([market, d], axis=1)

market2 = market.copy()

"""
mark_date = market2.isnull().sum(axis=1).index
mark_col = market2.isnull().sum(axis=0).index


#한 날짜에 850개 초과의 기업이 주가가 없다면 해당 날짜 제거 (공휴일이라 판단)
for i in mark_date:
  if market2.isnull().sum(axis=1)[i] > 850:
    market2.drop(i, inplace=True)

#한 기업이 40일 이상의 주가가 없으면 제거 (결측치가 너무 많으면 평균이나 선형보간법으로 결측치를 채우는게 무리가 있다고 판단)
for i in mark_col:
  if market2.isnull().sum(axis=0)[i] > 40:
    market2.drop(i, axis=1, inplace=True)

#선형 보간법으로 우선 채우고 연속된 결측치의 경우에는 전 값으로 결측치를 처리
market2.interpolate(method='linear', inplace=True)
market2.fillna(method='bfill', inplace=True)
market2.drop('Pharnext SA', axis=1, inplace=True)
"""

In [ ]:
market2.drop('companyId', axis=1, inplace=True)
market2.interpolate(method='linear', inplace=True)
market2.fillna(method='bfill', inplace=True)
market2

In [ ]:
mark_col = market2.isnull().sum(axis=0).index
for i in mark_col:
  if market2.isnull().sum(axis=0)[i] > 40:
    market2.drop(i, axis=1, inplace=True)
market2

In [ ]:
import pandas as pd
financial_final = pd.read_csv("financial_final.csv")
estimates_final = pd.read_csv("estimates_final.csv")

In [ ]:
#Estimates 데이터에 대한 포트폴리오와 Financial 데이터에 대한 포트폴리오 그리고 주가 데이터 합치기

estimates_final2 = estimates_final.copy()
financial_final2 = financial_final.copy()
estimates_final2.set_index('Unnamed: 0', drop=True, inplace=True)
estimates_final2.index = estimates_final2.index.astype(str)
financial_final2.set_index('Unnamed: 0', drop=True, inplace=True)
financial_final2.index = financial_final2.index.astype(str)
financial_final2[list(financial_final2.columns)[1159:]]
result_df = pd.merge(estimates_final2, financial_final2[list(financial_final2.columns)[1159:]], left_index=True, right_index=True)
result_df

In [ ]:
result_df.to_csv("final_df.csv", index=True)

In [ ]:
import pandas as pd
final_df = pd.read_csv("final_df.csv")
final_df

In [ ]:
final_df.set_index("Unnamed: 0", drop=True, inplace=True)
final_df

In [ ]:
fin = final_df.copy()

col = list(fin.columns)
factors = ['Long-term Investments', 'Unearned Revenue Current, Total', 'Gain (Loss) On Sale Of Assets', 'EPS Normalized - Standard Deviation']

In [ ]:
from tqdm import tqdm
import statsmodels.api as sm
cnt = 0

for col in tqdm(fin.columns[:-132]):
  res = sm.OLS(endog=fin[col], exog=sm.add_constant(fin[factors])).fit()
  if res.pvalues[1] <= 0.05:
    cnt += 1

In [ ]:
print(cnt)

In [ ]:
#기존에는 Estimates와 Financial 각각에서의 프레임워크가 존재했음
#Estimate는 연간 데이터로 예를 들어 22년 예상치를 통해 22년의 포트폴리오를 구성함
#Financial는 분기 데이터로 예를 들어 22년 1분기 재무 데이터를 통해 22년 2분기 포트폴리오를 구성함
#Estimate에서의 유의미한 팩터와 Financial에서의 유의미한 팩터를 합쳐서 팩터 투자를 진행할거임
#재무 팩터 같은 경우는 분기마다 업데이트 되며, 예상치 팩터는 연간 별로 업데이트가 된다
#-> 따라서 예상치는 1년 동안 같은 값을 가지는 반면에 재무 팩터는 분기마다 업데이트가 된다
#멀티 팩터 모델에서 동일한 가중치를 주어 한 데이터에 대해 Normalize를 진행 후
#이 값들을 멀티 팩터에 대해 다 종합해서 더한 후에 거기서 크기 순으로 순위를 결정하고 상위 20개 종목에 롱을 하는 투자 전략

def cal(item, year, quarter):
  #재무 데이터면 분기별로 데이터 불러오기
  if item in financial_item:
    example_feature = df_finance[(df_finance['dataItemName'] == item)  & (df_finance['fiscalYear'] == year) & (df_finance['fiscalQuarter'] == quarter)]
    example = example_feature[['companyId', 'dataItemValue', 'ISOCode']].copy()

  #재무가 아닌 예상치 데이터라면 연간 데이터 불러오기
  else:
  #22년 4분기 재무 데이터를 통해서 23년 1분기의 포트폴리오를 구성하는데, 예상치는 23년 데이터를 통해 23년의 포트폴리오를 구성한다
    if (quarter == 4):#예외 처리
      year = 2023
    example_feature = df_est[(df_est['dataItemName'] == item)  & (df_est['fiscalYear'] == year)]
    example = example_feature[['companyId', 'dataItemValue', 'ISOCode']].copy()

  example2 = example.copy()

  for id in list(map(int, example2['companyId'])):
    num = len(example2[example2['companyId']==id]['dataItemValue'].values)
    if num > 1:
      idx = list(example2[example2['companyId']==id]['dataItemValue'].index)
      avg = []
      for i in idx:
        a = example2.loc[i]['dataItemValue']
        avg.append(a)
      average = sum(avg) / len(avg)
      example2.loc[idx[0]]['dataItemValue'] = average
      example2.drop([i for i in idx[0:]], inplace=True)
  example2.reset_index(drop=True, inplace=True)

  #재무 데이터가 어느 나라의 기업이냐에 따라 명시되어 있는 Currency가 다름 (ISOCode를 통해 확인 가능)
  #이런 식으로 각 포트폴리오 구성이 시작되는 날짜를 기준으로 환율을 곱하여 값을 맞춰줌

  #본래 재무 데이터는 전 분기 데이터를 가지고 다음 분기의 종목을 결정하고, 예상치 데이터는 해당 년도의 예상치를 바탕으로 해당 년도의 종목의 결정함
  #하지만 23년 재무 데이터가 없으므로 23년 분기마다 새로운 종목을 구성하는게 불가능하다
  #따라서 22년 4분기 재무 데이터를 바탕으로 23년 9월까지의 종목을 한 번에 선정
  if item in financial_item:
    if (year == 2022) & (quarter == 2):
      for i in range(len(example2)):
        if example2['ISOCode'][i] != "USD":
          example2['dataItemValue'][i] = (example2['dataItemValue'][i]) / (fx.loc["2022-09-22", example2['ISOCode'][i]])
    elif (year == 2022) & (quarter == 3):
      for i in range(len(example2)):
        if example2['ISOCode'][i] != "USD":
          example2['dataItemValue'][i] = (example2['dataItemValue'][i]) / (fx.loc["2022-10-01", example2['ISOCode'][i]])
    else:
      #year = 2022, quarter = 4; year = 2023을 둘 다 고려
      for i in range(len(example2)):
        if example2['ISOCode'][i] != "USD":
          example2['dataItemValue'][i] = (example2['dataItemValue'][i]) / (fx.loc["2023-01-01", example2['ISOCode'][i]])
  else:
    if (year == 2022):
      for i in range(len(example2)):
        if example2['ISOCode'][i] != "USD":
          example2['dataItemValue'][i] = (example2['dataItemValue'][i]) / (fx.loc["2022-09-22", example2['ISOCode'][i]])
    else:
      for i in range(len(example2)):
        if example2['ISOCode'][i] != "USD":
          example2['dataItemValue'][i] = (example2['dataItemValue'][i]) / (fx.loc["2023-01-01", example2['ISOCode'][i]])


  #작업하는 데이터의 값을 내림차순으로 정렬
  example2.drop('ISOCode', axis=1, inplace=True)
  example2.sort_values(by='dataItemValue', ascending=False, inplace=True)

  #Initialize the StandardScaler
  scaler = StandardScaler()

  # Apply standard scaling to the chosen column
  example2.set_index('companyId', drop=True, inplace=True)
  scaled = scaler.fit_transform(example2)
  scaled_df = pd.DataFrame(scaled, index = example2.index, columns=example2.columns)

  return scaled_df

In [ ]:
pd.options.mode.chained_assignment = None
from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler

quar = [2,3,4]
y = 2022

#팩터가 재무라면 분기별로 (23년은 데이터가 없어서 22년 4분기 데이터를 활용하여 23년 9월까지의 종목 선정),
#예상치라면 연간별로 값을 산출하여 포트폴리오를 새롭게 구성함

def weights(items):
  result_df = pd.DataFrame()
  fin_df = pd.DataFrame()
  for q in quar:
    for item in items:
      scaled_df = cal(item, y, q)
      # 22년 2분기 재무 데이터로 9월 22일 ~ 9월 31일 포폴 가중치 계산
      result_df = pd.concat([result_df, scaled_df]).groupby(level=0).sum()
    result_df.sort_values('dataItemValue', ascending=False, inplace=True)
    comp_id = list(result_df.index)
    market3 = market2.copy()
    market3 = market3.reindex(columns=comp_id)
    market3 = market3.dropna(axis=1)

    if (q == 2):
        price_y = market3.loc['2022-09-22':'2022-09-30'].copy()
    elif (q == 3):
        price_y = market3.loc['2022-10-01':'2022-12-31'].copy()
    else:
        price_y = market3.loc['2023-01-01':'2023-09-22'].copy()
    for col in price_y.columns:
      price_y[col] = price_y[col].apply(lambda x: float(x))

    price_y = price_y.pct_change(1).dropna()
    price_y = price_y + 1
    cum_ret = price_y.cumprod() - 1
    cum_ret["Portfolio"] = (cum_ret.iloc[:, 0:200].sum(axis=1)) / 200
    fin_df = pd.concat([fin_df, cum_ret["Portfolio"]], axis=0)
  return fin_df

In [ ]:
qqq = pd.read_csv("test_result.csv")
qqq.drop('Unnamed: 0', axis=1, inplace=True)
qty = qqq.copy()

In [ ]:
qty

In [ ]:
unique_id = list(qty['companyName'].unique())
kt = pd.DataFrame()

#팩터 모델링을 위한 Training의 기간은 21년 9월 22일부터 22년 9월 22일까지
for id in unique_id:
  ex = qty[(qty['companyName'] == id)].copy()
  ex.rename(columns={'new' : id}, inplace=True)
  ex.drop(['companyName'], axis=1, inplace=True)
  ex.sort_values(by='announcedDate', inplace=True)
  ex.set_index('announcedDate', inplace=True, drop=True)
  ex = ex[~ex.index.duplicated(keep='first')]
  ex = ex.apply(pd.to_numeric, errors='coerce')
  kt = pd.concat([kt, ex], axis=1)

kt2 = kt.copy()
"""
mark_date = market2.isnull().sum(axis=1).index
mark_col = market2.isnull().sum(axis=0).index

#한 날짜에 850개 초과의 기업이 주가가 없다면 해당 날짜 제거 (공휴일이라 판단)
for i in mark_date:
  if market2.isnull().sum(axis=1)[i] > 850:
    market2.drop(i, inplace=True)

#한 기업이 40일 이상의 주가가 없으면 제거 (결측치가 너무 많으면 평균이나 선형보간법으로 결측치를 채우는게 무리가 있다고 판단)
for i in mark_col:
  if market2.isnull().sum(axis=0)[i] > 40:
    market2.drop(i, axis=1, inplace=True)

#선형 보간법으로 우선 채우고 연속된 결측치의 경우에는 전 값으로 결측치를 처리
market2.interpolate(method='linear', inplace=True)
market2.fillna(method='bfill', inplace=True)
market2.drop(39414143, axis=1, inplace=True)
"""

In [ ]:
kt2 = kt.copy()

In [ ]:
kt2.loc['2022-10-01'] += kt2.loc["2022-09-23":"2022-09-30"].sum()
kt2.loc['2023-01-01'] += kt2.loc["2022-10-01":"2022-12-31"].sum()
kt2

In [ ]:
index_1 = list(kt2.loc['2022-09-22'].sort_values(ascending=False).index)
index_2 = list(kt2.loc['2022-10-01'].sort_values(ascending=False).index)
index_3 = list(kt2.loc['2023-01-01'].sort_values(ascending=False).index)

In [ ]:
len(index_3)

1300

In [ ]:
lis = [index_1, index_2, index_3]

In [ ]:
market3

In [ ]:
for i in lis:
  print(i)

In [ ]:
fin_df = pd.DataFrame()

for i in lis:
  market3 = market2.copy()
  market3 = market3.reindex(columns=i)

  #선형 보간법으로 우선 채우고 연속된 결측치의 경우에는 전 값으로 결측치를 처리
  market3.interpolate(method='linear', inplace=True)

  if i == index_1:
    price_y = market3.loc['2022-09-22':'2022-09-30'].copy()
  elif i == index_2:
    price_y = market3.loc['2022-10-01':'2022-12-31'].copy()
  else:
    price_y = market3.loc['2023-01-01':'2023-09-22'].copy()

  for col in price_y.columns:
    price_y[col] = price_y[col].apply(lambda x: float(x))

  price_y = price_y.pct_change(1).dropna()
  price_y = price_y + 1
  cum_ret = price_y.cumprod() - 1
  cum_ret["Portfolio"] = (cum_ret.iloc[:, 0:200].sum(axis=1)) / 200
  fin_df = pd.concat([fin_df, cum_ret["Portfolio"]], axis=0)

In [ ]:
fin_df

In [ ]:
import matplotlib.pyplot as plt
# Plot the DataFrame with legends
ax = fin_df.plot(linestyle='-', figsize=(8, 5))

# Set labels and title
ax.set_xlabel("Dates")
ax.set_ylabel("Cumulative Returns (%)")
ax.set_title("Factor investing VS IBB")

# Display the legend using the column names
ax.legend(df.columns)

# Show the plot
plt.show()

In [ ]:
import yfinance as yf
import pandas as pd

# Define the date range
start_date = '2022-09-22'
end_date = '2023-09-22'

# Download historical data for S&P 500
IBB = yf.download('IBB', start=start_date, end=end_date)

# Extract the closing prices
IBB_close = IBB['Close']

# Calculate daily returns
IBB_returns = IBB_close.pct_change()

# Calculate total return
total_return = ((1 + IBB_returns).cumprod() - 1).dropna()
total_return

In [ ]:
factor = ['Long-term Investments', 'Unearned Revenue Current, Total', 'Gain (Loss) On Sale Of Assets', 'EPS Normalized - Standard Deviation']
q = weights(factor)
q

In [ ]:
z = q.copy()
df = pd.concat([z, total_return], axis=1)
df.dropna(inplace=True)
new_column_names = {
    0: factor[0],
    'Close': 'IBB',
}

df.rename(columns=new_column_names, inplace=True)
df

In [ ]:
# Plot the DataFrame with legends
ax = df.plot(linestyle='-', figsize=(8, 5))

# Set labels and title
ax.set_xlabel("Dates")
ax.set_ylabel("Cumulative Returns (%)")
ax.set_title("Factor investing VS IBB")

# Display the legend using the column names
ax.legend(df.columns)

# Show the plot
plt.show()